In [14]:
pip install streamlit


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ---------------------------------------- 9.8/9.8 MB 55.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   --------------------------------------- 731.2/731.2 kB 10.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   -------------------- ------------------- 13.1/25.3 MB 68.7 MB/s eta 0:00:01
   ---------------------------------------  25.2/25.3 MB 66.4 MB/s eta 0:00:01
   ---------------------------------------- 25.3/25.3 MB 61.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 53.1 MB/s eta 0:00:00
Using cached jsonschema-4.

In [ ]:
import requests
import re

# Replace with your actual Etherscan API key
ETHERSCAN_API_KEY = "EVWY88Y9UDYU4JYTBFHRN7WNPVA253YRTA"

def is_ethereum_address(address):
    """Checks if the given address is a valid Ethereum address."""
    return bool(re.match(r"^0x[a-fA-F0-9]{40}$", address))

def is_bitcoin_address(address):
    """Checks if the given address is a valid Bitcoin address."""
    return bool(re.match(r"^[13][a-km-zA-HJ-NP-Z1-9]{25,34}$", address))

def check_bitcoin_scam_activity(address):
    """Fetches Bitcoin transactions and detects scam-related activity."""
    url = f"https://blockchain.info/rawaddr/{address}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        total_inputs = 0
        total_outputs = 0
        large_transactions = 0
        high_recipient_txs = 0

        HIGH_RECIPIENT_THRESHOLD = 10  # More than 10 recipients
        LARGE_TX_THRESHOLD = 1_000_000  # 1 BTC in satoshis

        for tx in data.get("txs", []):
            inputs = len(tx.get("inputs", []))
            outputs = len(tx.get("out", []))

            total_inputs += inputs
            total_outputs += outputs

            if outputs > HIGH_RECIPIENT_THRESHOLD:
                high_recipient_txs += 1

            for output in tx.get("out", []):
                if output.get("value", 0) > LARGE_TX_THRESHOLD:
                    large_transactions += 1

        if total_inputs == 0 and total_outputs == 0:
            print(f"🔍 BTC Address {address}: No transactions found.")
            return

        print(f"\n🔍 BTC Address: {address}")
        print(f"   📥 Total Inward Transfers: {total_inputs}")
        print(f"   📤 Total Outward Transfers: {total_outputs}")
        print(f"   💰 Large Transactions (>1 BTC): {large_transactions}")
        print(f"   🔀 Transactions with Many Recipients (>10): {high_recipient_txs}")

        if high_recipient_txs > 2 or large_transactions > 2:
            print("   ⚠️ Potential Scam Detected!")
        else:
            print("   ✅ Legitimate Activity Detected.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for BTC address {address}: {e}")

def check_ethereum_scam_activity(address):
    """Fetches Ethereum transactions and detects scam-related activity."""
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={ETHERSCAN_API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if data.get("status") != "1":
            print(f"🔍 ETH Address {address}: No transactions found or invalid address.")
            return

        transactions = data.get("result", [])
        total_inward = 0
        total_outward = 0
        large_transactions = 0
        high_recipient_txs = 0

        HIGH_RECIPIENT_THRESHOLD = 10  # More than 10 recipients
        LARGE_TX_THRESHOLD = 1 * 10**18  # 1 ETH in Wei

        for tx in transactions:
            from_address = tx.get("from", "").lower()
            to_address = tx.get("to", "").lower()
            value = int(tx.get("value", 0))

            if to_address == address.lower():
                total_inward += 1
            if from_address == address.lower():
                total_outward += 1

                if value > LARGE_TX_THRESHOLD:
                    large_transactions += 1

        if total_inward == 0 and total_outward == 0:
            print(f"🔍 ETH Address {address}: No transactions found.")
            return

        print(f"\n🔍 ETH Address: {address}")
        print(f"   📥 Total Inward Transfers: {total_inward}")
        print(f"   📤 Total Outward Transfers: {total_outward}")
        print(f"   💰 Large Transactions (>1 ETH): {large_transactions}")
        print(f"   🔀 Transactions with Many Recipients (>10): {high_recipient_txs}")

        if large_transactions > 2 or high_recipient_txs > 2:
            print("   ⚠️ Potential Scam Detected!")
        else:
            print("   ✅ Legitimate Activity Detected.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for ETH address {address}: {e}")

if __name__ == "__main__":
    user_input = input("Enter Bitcoin or Ethereum address(es) (comma-separated for multiple): ")
    addresses = [addr.strip() for addr in user_input.split(",")]

    for address in addresses:
        if is_bitcoin_address(address):
            check_bitcoin_scam_activity(address)
        elif is_ethereum_address(address):
            check_ethereum_scam_activity(address)
        else:
            print(f"❌ Invalid address format: {address}")



🔍 ETH Address: 0x0000C5d73291e9ae226E7fE59ea1e6b68bE30000
   📥 Total Inward Transfers: 5022
   📤 Total Outward Transfers: 0
   💰 Large Transactions (>1 ETH): 0
   🔀 Transactions with Many Recipients (>10): 0
   ✅ Legitimate Activity Detected.


Storing of Scam Bitcoin Addresses from forumns into 'scam_bitcoin_addresses'

In [13]:
import requests
import re

# URL of the text file
url = "https://raw.githubusercontent.com/mitchellkrogza/Badd-Boyz-Bitcoin-Scammers/master/bitcoin-scammers.txt"

# Fetch the text content
response = requests.get(url)
if response.status_code == 200:
    text = response.text
else:
    raise Exception(f"Failed to fetch file: {response.status_code}")

# Regular expression pattern for Bitcoin addresses
btc_pattern = re.compile(r'\b[13][a-km-zA-HJ-NP-Z1-9]{25,34}\b')

# Extract all matches
scam_bitcoin_addresses = btc_pattern.findall(text)

# Remove duplicates by converting the list to a set, then back to a list
scam_bitcoin_addresses = list(set(scam_bitcoin_addresses))

# Optional: Sort the list
scam_bitcoin_addresses.sort()

# Output the number of addresses found
print(f"Found {len(scam_bitcoin_addresses)} unique Bitcoin addresses.")

# Example: Print the first 10 addresses
print(scam_bitcoin_addresses[:10])


Found 85 unique Bitcoin addresses.
['12ziVv4aQkZTA1gj86Y9uYQByG4CcdVcTA', '135qVXXBZb3v2tQcLJRA8UAndiUYNybh3J', '13DAd45ARMJW6th1cBuY1FwB9beVSzW77R', '13cyEdT7kyH2f4j9xchvDGhv1o64MYNLUS', '14DvFghvkzQujf5Kd5AL2VKjxaYm5KidxR', '14MDSpmw9B7MVeh3d99TL8T9nc5d1PMYGZ', '14VbGhtysr6wrNs7EhdKiS2NoMmMDBM8cn', '14bXUoPwruptLamUfKTuMW39Qy1q4ohX9w', '15WG3a68ZDPsYyJUkMKQkqwkvWykT74ufB', '15ZHnf1MPn6ybb8yUeAoCQ1AJtiKhg3NrP']
